In [ ]:
print('hello')

In [ ]:
# downloading the libraries

import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)

import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap # correlation matrix

# worldcloud
import random
from wordcloud import WordCloud

# display ERD
from IPython.display import Image



# I - Exploring the data

objective is to get more familiar with the data and review pandas / visulization logic

## A - Fetching the data

In [ ]:
# understanding where the files are located with terminal operations

!cd

In [ ]:
# adding the path to fetch the files

raw_path = '../albert-hod-shared/notebooks'


In [ ]:
# creating pandas dataframes from .csv

df_customer = pd.read_csv(f'../data/olist_datasets/olist_customers_dataset.csv')
df_geolocation = pd.read_csv(f'../data/olist_datasets/olist_geolocation_dataset.csv')
df_orders = pd.read_csv(f'../data/olist_datasets/olist_orders_dataset.csv')
df_order_items = pd.read_csv(f'../data/olist_datasets/olist_order_items_dataset.csv')
df_order_payments = pd.read_csv(f'../data/olist_datasets/olist_order_payments_dataset.csv')
df_order_reviews = pd.read_csv(f'../data/olist_datasets/olist_order_reviews_dataset.csv')
df_products = pd.read_csv(f'../data/olist_datasets/olist_products_dataset.csv')
df_sellers = pd.read_csv(f'../data/olist_datasets/olist_sellers_dataset.csv')


## B - Exploring the orders dataframe

In [ ]:
# printing columns, null count, and dtypes for the orders dataframe

df_orders.info()


In [ ]:
# printing the first 10 lines of the orders dataframe

df_orders.head(10)

# the last 10 lines of the dataframe

df_orders.tail(10)

# a sample of ten lines

df_orders.sample(10)

# getting all null values

df_orders.isnull().sum()


**why are there null values for approved_at, delivered_carrier_date, delivered_customer_date?**

*-> answer: because some orders are not approved/delivered yet

In [ ]:
# fetching only particular columns of the df_orders dataframe

## fetch the order_approved_at columns in a data series format, then in a dataframe format, and print the first five lines

df_orders_approved_at = df_orders[['order_approved_at']]

df_orders_approved_at.head(5)

## fetch the order_id, order_status, approved_at, delivered_carrier_date, delivered_customer_date

df_test = df_orders[['order_approved_at','order_id','order_status','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date']]

In [ ]:
# fetching the approved_at null columns - create another dataframe df_filtered, which will be filtered on order_approved_at is null, and print first 10 rows

df_filtered = df_orders[df_orders['order_approved_at'].isnull()]
df_filtered.head()


In [ ]:
# count the order_id with a null approved_at by order_status

# with groupby method

df_grouped = df_filtered.groupby('order_status').agg(
    order_count=('order_id', 'count'),
    purchase_min=('order_purchase_timestamp', 'min')
)

df_grouped

# with value_counts method
df_status_counts = df_filtered['order_status'].value_counts().reset_index()
df_status_counts.columns = ['order_status', 'order_count']

df_status_counts.head()



**what do you think of the results? Why is there a null approved at to begin with?**

-> *answer*: Most null approved at are from canceled/uncompleted orders. However, the 14 delivered are likely due to data issues.

## C - Creating a simple visualization using Matplotlib or Seaborn

In [ ]:
# Creating a simple plot with matplotlib: count of non-approved orders by status

non_approved_orders = df_orders[df_orders['order_approved_at'].isnull()]

non_approved_counts = non_approved_orders.groupby('order_status')['order_id'].count()

fig, ax = plt.subplots(figsize=(12,6))

ax = plt.bar(non_approved_counts.index, non_approved_counts.values)

plt.xlabel('order_status')
plt.ylabel('count of non-approved orders')
plt.title('count of non-approved orders by status', fontsize=12, fontweight='bold')

plt.show()


In [ ]:
# Creating a simple plot with seaborn: count of non-approved orders by status

# Filter for non-approved orders
non_approved_orders = df_orders[df_orders['order_approved_at'].isnull()]

# Count the number of non-approved orders by status
non_approved_counts = non_approved_orders.groupby('order_status')['order_id'].count().reset_index()

# Create a bar plot using Seaborn
plt.figure(figsize=(12, 6))
sns.barplot(x='order_status', y='order_id', data=non_approved_counts)

# Set labels and title
plt.xlabel('Order Status')
plt.ylabel('Count of Non-Approved Orders')
plt.title('Count of Non-Approved Orders by Status', fontsize=12, fontweight='bold')

# Show the plot
plt.show()

## D - Exploring all dataframes at once

In [ ]:
# an interesting way to look at all dataframes at once, to quickly get what are the null columns

# Collections for each dataset
datasets = [df_customer, df_geolocation, df_orders, df_order_items, df_order_payments,
            df_order_reviews, df_products, df_sellers]
names = ['df_customer', 'df_geolocation', 'df_orders', 'df_order_items', 'df_order_payments',
         'df_order_reviews', 'df_products', 'df_sellers']

# Creating a DataFrame with useful information about all datasets
data_info = pd.DataFrame({})
data_info['dataset'] = names
data_info['n_rows'] = [df.shape[0] for df in datasets]
data_info['n_cols'] = [df.shape[1] for df in datasets]
data_info['null_amount'] = [df.isnull().sum().sum() for df in datasets]
data_info['qty_null_columns'] = [len([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]
data_info['null_columns'] = [', '.join([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]

data_info.style.background_gradient()


**What are the dataframes with the most null columns? What's your explanation?**

*->answer:* df_order_reviews has the most null columns by far, likely due to missing review comments.
df_orders and df_products also have missing data, possibly from incomplete order details and product attributes.

## E - Joining 2 dataframes - how many orders were created in SP state?

In [ ]:
# which dataframes do we need to join to answer the question? Explore the dataframes and look for a link between orders and state

df_customer.head()

In [ ]:
# Merge the 2 datasets in a new df_merged dataset, and print the first 5 rows

df_merged = df_orders.merge(df_customer[['customer_id','customer_state']], on='customer_id', how='left')
df_merged.head()

In [ ]:
# filter the dataframe to have only rows with state = SP, and print the first 5 lines

df_merged_filtered = df_merged[df_merged.customer_state == 'SP']
df_merged_filtered = df_merged[df_merged['customer_state'] == 'SP']
df_merged_filtered.head()

In [ ]:
# count the number of orders by state, using both groupby and value_counts

# with groupby method

# with value_counts method
df_merged_filtered['customer_state'].value_counts()

# II - Exploratory analysis

our objective is to a) give an overview of the business, b) understand what's driving the bad reviews

## A - Creating a uniform dataframe for our analysis

**Creating an ERD of the database**

- in an ERD diagram tool
- create one item per table
- list all columns
- list their type
- find the primary and foreign keys
- link the tables together

--> upload directly the ERD in the notebook

In [ ]:
# Displaying the image with Ipython Image module

## the image is in the data/ERD folder
filename = '../data/ERD/olist_erd.png'

Image(filename=filename)


In [ ]:
# Merging the dataframes using the ERD above

df = df_orders.merge(df_order_items, on='order_id', how='left')
df = df.merge(df_products, on='product_id', how='left')
df = df.merge(df_customer, on='customer_id', how='left')
# df = df.merge(df_sellers, on='seller_id', how='left')
df = df.merge(df_order_reviews, on='order_id', how='left')
# df = df.merge(df_geolocation, how='left',left_on='customer_zip_code_prefix',right_on='geolocation_zip_code_prefix')
# df = df.merge(df_order_payments, on='order_id', how='left')

df.head()


In [ ]:
# Cleaning the dataframe and creating additional columns that will be useful for further analysis

# Ensure timestamps are in datetime format
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'], errors='coerce')
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'], errors='coerce')
df['delivery_time'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days

# Create useful features from order_purchase_timestamp
df['year'] = df['order_purchase_timestamp'].dt.year                   # year as integer
df['month'] = df['order_purchase_timestamp'].dt.month                  # month as integer
df['week'] = df['order_purchase_timestamp'].dt.isocalendar().week      # week in iso format (1, 2, 3 etc.)
df['day_of_week_int'] = df['order_purchase_timestamp'].dt.weekday + 1  # day of week as integer (1 = Monday, etc.)
df['hour'] = df['order_purchase_timestamp'].dt.hour                   # hour of day
df['year_month'] = df['order_purchase_timestamp'].dt.strftime('%Y-%m')  # year and month, just like 2017-10

df.head()



## B - Looking at Reviews

### 1 - Let's look at the distribution of the review score

In [ ]:
# Create a distribution plot (dist plot) with seaborn on the review score

# Create a bar plot for the review_score column
sns.countplot(data=df, x='review_score')

# Add labels and a title
plt.title('Distribution of Review Scores')
plt.xlabel('Review Score')
plt.ylabel('Frequency')

# Show the plot
plt.show()

**What do you make of those results?**

*->answer:* The majority of reviews are positive, with 4.0 and 5.0 scores making up 55.6% of the total, indicating general satisfaction. The quantity of reviews grows with satisfaction. However, there are slightly more 1/5 scores than 2/5 and 3/5, reflecting the tendency of some customers to be polarized, often representing dissatisfied or angry customers.

### 2 - Let's look at the customer verbatims

In [ ]:
# Create a wordcloud on orders with negative reviews only

# Filter the DataFrame for negative reviews (1.0 and 2.0)
negative_reviews = df[df['review_score'].isin([1.0, 2.0])]

# Join all the review messages into one large string
text = ' '.join(negative_reviews['review_comment_message'].dropna())

# Create the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

**What are the first reasons you see behind a bad review? What could we do to improve this wordcloud?**

*-> answer:* poor quality, bad support and delivery issues

### 3 - Let's create a correlation matrix and identify other reasons behind bad reviews

In [ ]:
# let's isolate the columns we'll use for our correlation matrix
columns_to_include = ['price', 'freight_value', 'product_weight_g', 'product_length_cm',
                      'product_height_cm', 'product_width_cm', 'delivery_time', 'review_score']

# Create a new DataFrame with only these columns
df_correlation = df[columns_to_include]

# print all the columns and their types using the method of your choice
df_correlation

In [ ]:
# bonus: can we create additional features?
# example: create a is_delivered column?


In [ ]:
# Create the correlation matrix based on the columns you want

# Select relevant features for correlation analysis

colors = ['#001F3F', '#0074D9', '#7FDBFF', '#39CCCC', '#3D9970', '#2ECC40', '#01FF70']

correlation_features = df_correlation

# Calculate the correlation matrix
correlation = correlation_features.corr()

# use seaborn to print the correlation matrix
plt.figure(figsize=(10, 8))  # Adjust the size of the plot if necessary
sns.heatmap(correlation, annot=True, cmap=sns.diverging_palette(20, 220, as_cmap=True),
            vmin=-1, vmax=1, fmt='.2f', linewidths=0.5, center=0)

plt.title('Correlation Matrix')
plt.show()

**What are the correlations you see?**

*-> answer:* The delivery time has a negative correlation with the review score, and is_delivered has a  positive correlation with the review score

## C - Looking at orders

### 1 - Orders by statuses

In [ ]:
# Create a visualization to count the orders by statuses

plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='order_status', hue='order_status', palette='Set2', legend=False)

plt.title('Number of Orders by Status')
plt.xlabel('Order Status')
plt.ylabel('Count of Orders')

plt.xticks(rotation=45)
plt.show()

### 2 - Evolution of total orders over time

In [ ]:
# Create a visualization to see the evolution of orders over time

# Group by date and count the number of orders
orders_over_time = df.groupby(df['order_purchase_timestamp'].dt.date).size()

# Create the time series plot
plt.figure(figsize=(12, 6))
sns.lineplot(x=orders_over_time.index, y=orders_over_time.values, color='blue')

plt.title('Evolution of Orders Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Orders')

plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### 3 - Comparing 2017 and 2018 (bonus)

In [ ]:
# Create a visualization to compare 2017 and 2018 on the same graph (bar charts with a "hue" = 'year'

# Ensure 'year' is a column in the DataFrame
df['year'] = pd.to_datetime(df['order_purchase_timestamp']).dt.year

# Filter data for 2017 and 2018
df_filtered = df[df['year'].isin([2017, 2018])]

# Create the bar plot with 'hue' set to 'year'
plt.figure(figsize=(10, 6))
sns.countplot(data=df_filtered, x='month', hue='year', palette='Set2')

# Add labels and title
plt.title('Comparison of Orders in 2017 vs 2018')
plt.xlabel('Month')
plt.ylabel('Number of Orders')

# Show the plot
plt.tight_layout()
plt.show()

## D - Delivery times

### 1 - Delivery time by product category

In [ ]:
# Start by counting the orders by product category, and create an 'Other section' for product categories that have the lowest amount of orders

# Count the occurrences of each product category
category_counts = df['product_category_name'].value_counts()

# Only keep the product category that are above a certain threshold, put the others in a "Other' category
threshold = 3000

df['product_category_name_filtered'] = df['product_category_name'].apply(
    lambda x: x if pd.notna(x) and category_counts.get(x, 0) >= threshold else 'Other'
)

df['product_category_name_filtered'].value_counts()


In [ ]:
# Create a boxplot visualization that shows the delivery time by product category

plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='product_category_name_filtered', y='delivery_time')

plt.xticks(rotation=45)

plt.title('Delivery Time by Product Category', fontsize=16)
plt.xlabel('Product Category', fontsize=12)
plt.ylabel('Delivery Time', fontsize=12)

plt.tight_layout()
plt.show()

### 2 - Delivery time by day of the week

In [ ]:
# Create a violin box plot visualization to see the delivery time by day of the week
plt.figure(figsize=(10, 6))
sns.violinplot(data=df, x='day_of_week_int', y='delivery_time', hue='day_of_week_int', palette='Set2')

plt.title('Delivery Time by Day of the Week', fontsize=16)
plt.xlabel('Day of the Week', fontsize=12)
plt.ylabel('Delivery Time', fontsize=12)

plt.xticks([0, 1, 2, 3, 4, 5, 6], ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

plt.tight_layout()
plt.show()

### 3 - Average delivery time by month  (bonus)

In [ ]:
# Create barplot for the monthly average delivery time


## E - Bonus

### 1 - Order distribution by day of week and time of day

In [ ]:
# Count plot showing order distribution by day of week


In [ ]:
# Count plot showing order distribution by time of day


### 2 - Payments

In [ ]:
# Check the dataframe


In [ ]:
# Create a donut (pie chart) that show transactions by payment types


In [ ]:
# Show the evolution of payment types over time


### 3 - Geolocation

In [ ]:
# Create a heatmap of all orders by location using an API to retrieve the latitude and longitude of every location, and display them on a map


### 4 - Profit computation

In [ ]:
# Show the monthly profit evolution for the best 4 product categories
# x axis: months
# y axis: gross profit (payment value - total order value)
# hue : top product categories
